# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv file created in previous step
output_data_file = "../output_data/cities.csv"
weather_df = weather_df = pd.read_csv(output_data_file)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
## Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidty
humidity = weather_df["Humidity"]

# Create the figure for the heatmap
figure_layout = {
    'width': '1200px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=(10,0),zoom_level=2)


# Create heat layer based on humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create ideal weather DataFrame
ideal_weather_df = weather_df.loc[(weather_df["Cloudiness"] == 0) & 
                                  (weather_df["Wind Speed"] < 10) & 
                                  (weather_df["Max Temp"] > 70) &
                                  (weather_df["Max Temp"] < 80)]

# Remove rows with missing information
ideal_weather_df = ideal_weather_df.dropna()


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a dataframe for hotel data by copying the ideal weather dataframe
hotel_df = ideal_weather_df.copy()

# Add column Hotel Name to the dataframe
hotel_df["Hotel Name"] = ""

# Reset the index
hotel_df.reset_index(drop=True,inplace=True)

# Configure search criteria
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# Set up parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# Set the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each city
for index,row in hotel_df.iterrows():
    # Add paramater location using the Lat/Lng
    params['location'] = (f"{row['Lat']},{row['Lng']}")

    # Send a request to the API
    response = requests.get(base_url, params=params).json()

    # Read the response and try to get the hotel name, if no hotels set it to blanks
    try:
        # get first hotel name
        hotel_name = response["results"][0]["name"]
    except IndexError:
        hotel_name=""

    # Add hotel name to dataframe row
    hotel_df.loc[index, "Hotel Name"] = hotel_name

# Remove cities that did not return a hotel
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Configure markers using the locations for the marker and the Hotel data to display when clicked
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…